<a href="https://colab.research.google.com/github/zanebookbinder/NFL-Betting/blob/main/Import_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/CS Projects/NFL Betting/spreadspoke_scores.csv'
df = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = df[(~df['team_favorite_id'].isna()) & (~df['over_under_line'].isna())]
df = df[df['schedule_season'] > 1987]
len(df)

8691

In [4]:
df['team_home'] = df['team_home'].replace('Phoenix Cardinals','Arizona Cardinals')
df['team_away'] = df['team_away'].replace('Phoenix Cardinals','Arizona Cardinals')

df['team_home'] = df['team_home'].replace('San Diego Chargers','Los Angeles Chargers')
df['team_away'] = df['team_away'].replace('San Diego Chargers','Los Angeles Chargers')

df['team_home'] = df['team_home'].replace('St. Louis Rams','Los Angeles Rams')
df['team_away'] = df['team_away'].replace('St. Louis Rams','Los Angeles Rams')

df['team_home'] = df['team_home'].replace('Boston Patriots','New England Patriots')
df['team_away'] = df['team_away'].replace('Boston Patriots','New England Patriots')

df['team_home'] = df['team_home'].replace('Los Angeles Raiders','Las Vegas Raiders')
df['team_away'] = df['team_away'].replace('Los Angeles Raiders','Las Vegas Raiders')

df['team_home'] = df['team_home'].replace('Oakland Raiders','Las Vegas Raiders')
df['team_away'] = df['team_away'].replace('Oakland Raiders','Las Vegas Raiders')

df['team_home'] = df['team_home'].replace('Houston Oilers','Tennessee Titans')
df['team_away'] = df['team_away'].replace('Houston Oilers','Tennessee Titans')

df['team_home'] = df['team_home'].replace('Tennessee Oilers','Tennessee Titans')
df['team_away'] = df['team_away'].replace('Tennessee Oilers','Tennessee Titans')

df['team_home'] = df['team_home'].replace('Washington Redskins','Washington Football Team')
df['team_away'] = df['team_away'].replace('Washington Redskins','Washington Football Team')

In [5]:
path = '/content/drive/MyDrive/CS Projects/NFL Betting/nfl_teams.csv'
teams = pd.read_csv(path)

In [6]:
teams = teams[['team_name','team_id','team_conference', 'team_division', 'team_conference_pre2002','team_division_pre2002']]

In [7]:
df.head(1)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
4450,9/4/1988,1988,1,False,Buffalo Bills,13.0,10.0,Minnesota Vikings,MIN,-2.5,44,Ralph Wilson Stadium,False,68.0,11.0,85,NaN


In [8]:
teams.head(1)

,team_name,team_id,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,ARI,NFC,NFC West,NFC,NFC West


In [9]:
pre2002 = pd.merge(df[df['schedule_season'] < 2002], teams[['team_name', 'team_id', 'team_division_pre2002']], left_on='team_home', right_on='team_name', how='left')
pre2002 = pd.merge(pre2002, teams[['team_name', 'team_id', 'team_division_pre2002']], left_on='team_away', right_on='team_name', how='left')

post2001 = pd.merge(df[df['schedule_season'] >= 2002], teams[['team_name', 'team_id', 'team_division']], left_on='team_home', right_on='team_name', how='left')
post2001 = pd.merge(post2001, teams[['team_name', 'team_id', 'team_division']], left_on='team_away', right_on='team_name', how='left')

In [10]:
df2 = pd.concat([pre2002, post2001], axis=0).reset_index(drop=True)

In [11]:
df2['home_div'] = np.where(df2['team_division_x'].isna(), df2['team_division_pre2002_x'], df2['team_division_x'])
df2['away_div'] = np.where(df2['team_division_y'].isna(), df2['team_division_pre2002_y'], df2['team_division_y'])
df2 = df2.rename({'team_id_x': 'home_id', 'team_id_y': 'away_id'}, axis=1)
df2 = df2.drop(['team_name_x','team_division_pre2002_x','team_name_y','team_division_pre2002_y','team_division_x','team_division_y'],axis=1)

In [12]:
df2['total'] = df2['score_home'] + df2['score_away']

home_spreads = []
for i, row in df2.iterrows():
    if row['team_favorite_id'] == row['home_id']:
        home_spreads.append(row['spread_favorite'])
    else:
        home_spreads.append(row['spread_favorite'] * -1)

df2['home_spread'] = home_spreads

In [13]:
df2.head(3)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,home_id,away_id,home_div,away_div,total,home_spread
0,9/4/1988,1988,1,False,Buffalo Bills,13.0,10.0,Minnesota Vikings,MIN,-2.5,44,Ralph Wilson Stadium,False,68.0,11.0,85,NaN,BUF,MIN,AFC East,NFC Central,23.0,2.5
1,9/4/1988,1988,1,False,Chicago Bears,34.0,7.0,Miami Dolphins,CHI,-3.5,46,Soldier Field,False,66.0,16.0,68,NaN,CHI,MIA,NFC Central,AFC East,41.0,-3.5
2,9/4/1988,1988,1,False,Cincinnati Bengals,21.0,14.0,Arizona Cardinals,CIN,-5.5,47,Cinergy Field,False,68.0,10.0,79,NaN,CIN,ARI,AFC Central,NFC West,35.0,-5.5


In [14]:
df2['playoff'] = ""
df2['week'] = 0
for i, row in df2.iterrows():
    if row['schedule_playoff'] == False:
        df2.at[i, 'playoff'] = 'Reg Season'

    else:
        df2.at[i, 'playoff'] = row['schedule_week']
    if not str(row['schedule_week']).isdigit():
        df2.at[i, 'week'] = -1

    else:
        df2.at[i, 'week'] = row['schedule_week']

In [15]:
df2['playoff'].replace('SuperBowl', 'Superbowl', inplace=True)
df2['playoff'].replace('WildCard', 'Wildcard', inplace=True)

In [16]:
df2.head(3)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,home_id,away_id,home_div,away_div,total,home_spread,playoff,week
0,9/4/1988,1988,1,False,Buffalo Bills,13.0,10.0,Minnesota Vikings,MIN,-2.5,44,Ralph Wilson Stadium,False,68.0,11.0,85,NaN,BUF,MIN,AFC East,NFC Central,23.0,2.5,Reg Season,1
1,9/4/1988,1988,1,False,Chicago Bears,34.0,7.0,Miami Dolphins,CHI,-3.5,46,Soldier Field,False,66.0,16.0,68,NaN,CHI,MIA,NFC Central,AFC East,41.0,-3.5,Reg Season,1
2,9/4/1988,1988,1,False,Cincinnati Bengals,21.0,14.0,Arizona Cardinals,CIN,-5.5,47,Cinergy Field,False,68.0,10.0,79,NaN,CIN,ARI,AFC Central,NFC West,35.0,-5.5,Reg Season,1


In [17]:
df2['over_under_line'] = df2['over_under_line'].astype(float)
df2['weather_humidity'] = df2['weather_humidity'].fillna(0)
df2['weather_humidity'] = df2['weather_humidity'].astype(int)

In [18]:
df2 = df2.fillna(0)

In [19]:
df2['home_margin'] = df2['score_home'] - df2['score_away']

In [20]:
df2.columns

Index(['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff',
       'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium',
       'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
       'weather_humidity', 'weather_detail', 'home_id', 'away_id', 'home_div',
       'away_div', 'total', 'home_spread', 'playoff', 'week', 'home_margin'],
      dtype='object')

In [21]:
df3 = df2[['schedule_date', 'schedule_season', 'week',  'playoff',
        'home_id', 'team_home', 'score_home', 'score_away', 'team_away', 'away_id', 'home_div', 'away_div', 
       'home_spread', 'home_margin', 'over_under_line', 'total',  'stadium',
       'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
       'weather_humidity', 'weather_detail' 
       ]]

In [22]:
df3 = df3.rename({'home_margin':'game_home_margin', 'total': 'game_total'},axis=1)

In [23]:
df3['home_div'] = df3['home_div'].map({'AFC East': 1, 'AFC North': 2, 'AFC West': 3, 'AFC South': 4, 'AFC Central': 5, 'NFC East': 6, 'NFC North': 7, 'NFC West': 8, 'NFC South': 9, 'NFC Central ': 10})
df3['away_div'] = df3['away_div'].map({'AFC East': 1, 'AFC North': 2, 'AFC West': 3, 'AFC South': 4, 'AFC Central': 5, 'NFC East': 6, 'NFC North': 7, 'NFC West': 8, 'NFC South': 9, 'NFC Central ': 10})

df3['playoff'] = df3['playoff'].map({'Reg Season': 1, 'Wildcard': 2, 'Division': 3, 'Conference': 4, 'Superbowl': 5})

df3['weather_detail'] = df3['weather_detail'].map({0: 0, 'DOME': 1, 'Rain': 2, 'DOME (Open Roof)': 3, 'Fog': 4, 'Rain | Fog': 4, 'Snow': 5, 'Snow | Fog': 5, 'Snow | Freezing Rain': 5})


df3['stadium_neutral'] = df3['stadium_neutral'].map({True : 1, False : 0})

In [24]:
for i, team in enumerate(list(df3['team_home'].unique())):
    df3['team_away'][df3['team_away'] == team] = i
    df3['team_home'][df3['team_home'] == team] = i

for i, stad in enumerate(list(df3['stadium'].unique())):
    df3['stadium'][df3['stadium'] == stad] = i

# for i in ['Pos1', 'Pos2', 'Tm1','Tm2']:
#     labels = all[i].unique().tolist()
#     mapping = dict(zip(labels,range(len(labels))) )
#     all.replace({i: mapping},inplace=True)

In [25]:
df3.head(2)

,schedule_date,schedule_season,week,playoff,home_id,team_home,score_home,score_away,team_away,away_id,home_div,away_div,home_spread,game_home_margin,over_under_line,game_total,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/4/1988,1988,1,1,BUF,0,13.0,10.0,18,MIN,1.0,NaN,2.5,3.0,44.0,23.0,0,0,68.0,11.0,85,0
1,9/4/1988,1988,1,1,CHI,1,34.0,7.0,24,MIA,NaN,1.0,-3.5,27.0,46.0,41.0,1,0,66.0,16.0,68,0


In [26]:
years = {}

for year in df3['schedule_season'].unique():
    years[year] = {}
    for team in df3['home_id'].unique():
        years[year][team] = ''

for i, row in df3.iterrows():
    if row['score_home'] > row['score_away']:
        years[row['schedule_season']][row['home_id']] += 'W'
        years[row['schedule_season']][row['away_id']] += 'L'

    if row['score_home'] < row['score_away']:
        years[row['schedule_season']][row['home_id']] += 'L'
        years[row['schedule_season']][row['away_id']] += 'W'

    if row['score_home'] == row['score_away']:
        years[row['schedule_season']][row['home_id']] += 'T'
        years[row['schedule_season']][row['away_id']] += 'T'


for year in years:
    for team in years[year]:
        teamyear = df3[(df3['schedule_season'] == year) & ((df3['home_id'] == team) | (df3['away_id'] == team))]
        weeks = list(teamyear['week'])
        if len(weeks) > 0:
            bye = [x for x in range(1, max(weeks)) if x not in weeks]
            if len(bye) > 0:
                bye = bye[0]
                years[year][team] = years[year][team][:bye] + ' ' + years[year][team][bye:]

In [27]:
def find_streak(s):
    s = s[::-1]
    
    if len(s) == 0:
        return 0
        
    if s[0] == 'W':
        count = 1
        for i in s[1:]:
            if i == 'W':
                count+=1
            else:
                return count

    if s[0] == 'L':
        count = -1
        for i in s[1:]:
            if i == 'L':
                count-=1
            else:
                return count

    if s[0] == 'T':
        return 0

In [28]:
df3['home_W'] = 0
df3['home_L'] = 0
df3['home_T'] = 0
df3['away_W'] = 0
df3['away_L'] = 0
df3['away_T'] = 0

df3['home_streak'] = 0
df3['away_streak'] = 0

for i, row in df3.iterrows():
    week = row['week'] - 1
    homes = years[row['schedule_season']][row['home_id']][:week]
    aways = years[row['schedule_season']][row['away_id']][:week]
    
    df3.at[i, 'home_W'] = homes.count('W')
    df3.at[i, 'home_L'] = homes.count('L')
    df3.at[i, 'home_T'] = homes.count('T')
    df3.at[i, 'home_streak'] = find_streak(homes)

    df3.at[i, 'away_W'] = aways.count('W')
    df3.at[i, 'away_L'] = aways.count('L')
    df3.at[i, 'away_T'] = aways.count('T')
    df3.at[i, 'away_streak'] = find_streak(aways)

In [29]:
df3['h_made_playoffs'] = 0
df3['h_last_year_w'] = 0

df3['a_made_playoffs'] = 0
df3['a_last_year_w'] = 0

for i, row in df3.tail(5).iterrows():
    hlastyear = df3[(df3['schedule_season'] == (row['schedule_season']-1)) & ((df3['home_id'] == row['home_id']) | (df3['away_id'] == row['home_id']))]

    if len(hlastyear) > 0:
        hmax = hlastyear[hlastyear['week'] == hlastyear['week'].max()].iloc[0]
        wonlast = 0

        if -1 in list(hlastyear['week']):
            df3.at[i, 'h_made_playoffs'] = hlastyear[hlastyear['week'] == -1]['playoff'].max()
        
        if hmax['home_id'] == row['home_id']:#team is home
            if hmax['score_home'] > hmax['score_away']:
                wonlast = 1

            df3.at[i, 'h_last_year_w'] = hmax['home_W'] + wonlast
        elif hmax['away_id'] == row['home_id']: #team is away
            if hmax['score_away'] > hmax['score_home']:
                wonlast = 1

            df3.at[i, 'h_last_year_w'] = hmax['away_W'] + wonlast

    alastyear = df3[(df3['schedule_season'] == (row['schedule_season']-1)) & ((df3['home_id'] == row['away_id']) | (df3['away_id'] == row['away_id']))]

    if len(alastyear) > 0:
        amax = alastyear[alastyear['week'] == alastyear['week'].max()].iloc[0]
        wonlast = 0

        if -1 in list(alastyear['week']):
            df3.at[i, 'a_made_playoffs'] = alastyear[alastyear['week'] == -1]['playoff'].max()
        
        if amax['home_id'] == row['away_id']:#team is home
            if amax['score_home'] > amax['score_away']:
                wonlast = 1

            df3.at[i, 'a_last_year_w'] = amax['home_W'] + wonlast
        elif amax['away_id'] == row['away_id']: #team is away
            if amax['score_away'] > amax['score_home']:
                wonlast = 1
                
            df3.at[i, 'a_last_year_w'] = amax['away_W'] + wonlast


In [30]:
df3['same_div'] = 0
df3['same_conf'] = 0

for i, row in df3.iterrows():
    if row['home_div'] == row['away_div']:
        df3.at[i, 'same_div'] = 1

    if row['home_div'] < 6 and row['away_div'] < 6:
        df3.at[i, 'same_conf'] = 1

    if row['home_div'] > 5 and row['away_div'] > 5:
        df3.at[i, 'same_conf'] = 1

In [31]:
df3['hPF'] = 21
df3['hPA'] = 21

df3['aPF'] = 21
df3['aPA'] = 21

for i, row in df3.iterrows():
    hprevious = df3[(df3['schedule_season'] == row['schedule_season']) & ((df3['home_id'] == row['home_id']) | (df3['away_id'] == row['home_id'])) & (df3['week'] < row['week']) & (df3['week'] > -1)]

    if len(hprevious) > 0:
        totalPF = 0
        totalPA = 0

        for i2, row2 in hprevious.iterrows():
            if row2['home_id'] == row['home_id']: #team is home
                totalPF += row2['score_home']
                totalPA += row2['score_away']

            if row2['away_id'] == row['home_id']: #team is away
                totalPF += row2['score_away']
                totalPA += row2['score_home']

        df3.at[i, 'hPF'] = totalPF / len(hprevious)
        df3.at[i, 'hPA'] = totalPA / len(hprevious)

    aprevious = df3[(df3['schedule_season'] == row['schedule_season']) & ((df3['home_id'] == row['away_id']) | (df3['away_id'] == row['away_id'])) & (df3['week'] < row['week'])  & (df3['week'] > -1)]

    if len(aprevious) > 0:
        totalPF = 0
        totalPA = 0

        for i2, row2 in aprevious.iterrows():
            if row2['home_id'] == row['away_id']: #team is home
                totalPF += row2['score_home']
                totalPA += row2['score_away']

            if row2['away_id'] == row['away_id']: #team is away
                totalPF += row2['score_away']
                totalPA += row2['score_home']

        df3.at[i, 'aPF'] = totalPF / len(aprevious)
        df3.at[i, 'aPA'] = totalPA / len(aprevious)


In [32]:
df3[df3['hPF'] == 1]

,schedule_date,schedule_season,week,playoff,home_id,team_home,score_home,score_away,team_away,away_id,home_div,away_div,home_spread,game_home_margin,over_under_line,game_total,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,home_W,home_L,home_T,away_W,away_L,away_T,home_streak,away_streak,h_made_playoffs,h_last_year_w,a_made_playoffs,a_last_year_w,same_div,same_conf,hPF,hPA,aPF,aPA
3204,9/30/2001,2001,3,1,WAS,21,13.0,45.0,7,KC,6.0,3.0,3.0,-32.0,35.5,58.0,41,0,57.0,17.0,59,0,0,2,0,0,2,0,NaN,NaN,0,0,0,0,0,0,1,33,13,20
4534,9/24/2006,2006,3,1,TB,12,24.0,26.0,29,CAR,9.0,9.0,3.0,-2.0,33.5,50.0,44,0,82.0,5.0,62,0,0,2,0,0,2,0,NaN,NaN,0,0,0,0,1,1,1,20,9,18


In [33]:
df3.fillna(0, inplace=True)

In [34]:
df3.head(3)

,schedule_date,schedule_season,week,playoff,home_id,team_home,score_home,score_away,team_away,away_id,home_div,away_div,home_spread,game_home_margin,over_under_line,game_total,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,home_W,home_L,home_T,away_W,away_L,away_T,home_streak,away_streak,h_made_playoffs,h_last_year_w,a_made_playoffs,a_last_year_w,same_div,same_conf,hPF,hPA,aPF,aPA
0,9/4/1988,1988,1,1,BUF,0,13.0,10.0,18,MIN,1.0,0.0,2.5,3.0,44.0,23.0,0,0,68.0,11.0,85,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,21,21,21,21
1,9/4/1988,1988,1,1,CHI,1,34.0,7.0,24,MIA,0.0,1.0,-3.5,27.0,46.0,41.0,1,0,66.0,16.0,68,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,21,21,21,21
2,9/4/1988,1988,1,1,CIN,2,21.0,14.0,22,ARI,5.0,8.0,-5.5,7.0,47.0,35.0,2,0,68.0,10.0,79,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,21,21,21,21


In [35]:
final = df3.drop(['schedule_date','home_id','away_id'],axis=1)

IDEAS:
4. pass offense and rush offense
5. pass defense and rush defense
7. over/under record
8. ATS record
9. margin?
10. coach experience
11. qb experience
12. H2H record?

https://www.pro-football-reference.com/teams/nwe/2021.htm
1. Get Coach page, Team stats --> offense and defense ranks, turnovers and turnovers forced, penalties for and against, Kicker XP% and FG%, preseason Superbowl and O/U odds (convert SB odds to implied probability)


In [36]:
final.to_csv('final.csv')